## Modelo TF-DF para obtener cursos similares basado en descripciones de curos
1. Este jupyter busca extaer lo que son las descripciones de todos los cursos ofg de la UC  mediante la API pública de OSUC

2. Ese jupyter entrena un model TF-DF para entregar cursos parecidos a otros basado en la descripción de los cursos

In [11]:
## librerías 
import requests
import ndjson
import pandas as pd
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:
'''
Obtener siglas de los cursos
'''

# Leer el archivo NDJSON
with open('courses-sections.ndjson', 'r', encoding='utf-8') as f:
    cursos_datos = ndjson.load(f)

# Filtrar: siglas con área no vacía
cursos_con_area = []

for curso in cursos_datos:
    area_curso = ""
    
    # Buscar la primera sección con área no vacía
    for section_key, section_data in curso.get('sections', {}).items():
        if section_data.get('area', '').strip():
            area_curso = section_data.get('area', '')
            break
    
    # Si encontramos área, agregar el curso
    if area_curso:
        cursos_con_area.append({
            'sigle': curso.get('sigle', '').lower(),  # Convertir a minúsculas
            'name': curso.get('name', ''),
            'area': area_curso
        })

# Crear DataFrame
df_cursos_filtrados = pd.DataFrame(cursos_con_area)

# Actualizar la lista CURSOS con las siglas en minúsculas
CURSOS = df_cursos_filtrados['sigle'].tolist()



       sigle                                               name  \
0    act1307  Comunicación Expresiva: Conciencia Vocal y Cor...   
1    act1308                  Conciencia y Expresión del Cuerpo   
2    act1309  Imaginarios para la Construcción del Teatro Ch...   
3    act1311  Mitos Clásicos y Creación de Relatos en la Dra...   
4    act1312  Performance Como Experiencia para la Transform...   
..       ...                                                ...   
289   vil630  Conozcamos Nuestra Fauna Silvestre: Identifica...   
290   vil633            Patrimonio Cultural y Memoria Histórica   
291   vil634                Wallmapu, Su Historia y Su Presente   
292   vil635                          Otra Cosa Es Con Guitarra   
293   vil636  Huerta para la Vida: la Huerta Agroecológica C...   

                               area  
0                             Artes  
1                             Artes  
2                             Artes  
3                             Artes  
4    

In [6]:
df_cursos_filtrados.head()

,sigle,name,area
0,act1307,Comunicación Expresiva: Conciencia Vocal y Cor...,Artes
1,act1308,Conciencia y Expresión del Cuerpo,Artes
2,act1309,Imaginarios para la Construcción del Teatro Ch...,Artes
3,act1311,Mitos Clásicos y Creación de Relatos en la Dra...,Artes
4,act1312,Performance Como Experiencia para la Transform...,Artes


In [7]:
df_cursos_filtrados

,sigle,name,area
0,act1307,Comunicación Expresiva: Conciencia Vocal y Cor...,Artes
1,act1308,Conciencia y Expresión del Cuerpo,Artes
2,act1309,Imaginarios para la Construcción del Teatro Ch...,Artes
3,act1311,Mitos Clásicos y Creación de Relatos en la Dra...,Artes
4,act1312,Performance Como Experiencia para la Transform...,Artes
...,...,...,...
289,vil630,Conozcamos Nuestra Fauna Silvestre: Identifica...,Ciencia y Tecnologia
290,vil633,Patrimonio Cultural y Memoria Histórica,Humanidades
291,vil634,"Wallmapu, Su Historia y Su Presente",Humanidades
292,vil635,Otra Cosa Es Con Guitarra,Artes


In [ ]:

# URL base de la API
BASE_URL = "https://buscaramos-v2-static-data.osuc.workers.dev/data"

def obtener_descripcion_curso(codigo_curso: str) -> Dict:
    """
    Hace un request a la API para obtener la descripción de un curso
    Retorna solo: sigle, name y description
    """
    try:
        url = f"{BASE_URL}/{codigo_curso}"
        response = requests.get(url, headers={"accept": "application/json"})
        response.raise_for_status()
        datos = response.json()
        
        # Solo voy a extraer los campos que necesito, no la response completa
        return {
            "sigle": datos.get("sigle"),
            "name": datos.get("name"),
            "description": datos.get("description")
        }
    except requests.exceptions.RequestException as e:
        print(f"Error obteniendo {codigo_curso}: {e}")
        return None

#
datos_cursos = []

for curso in CURSOS:
    print(f"Obteniendo datos del curso: {curso}")
    datos = obtener_descripcion_curso(curso)
    if datos:
        datos_cursos.append(datos)


Obteniendo datos del curso: act1307
Obteniendo datos del curso: act1308
Obteniendo datos del curso: act1309
Obteniendo datos del curso: act1311
Obteniendo datos del curso: act1312
Obteniendo datos del curso: act1313
Obteniendo datos del curso: act1314
Obteniendo datos del curso: act1315
Obteniendo datos del curso: act1318
Obteniendo datos del curso: act1319
Obteniendo datos del curso: act1321
Obteniendo datos del curso: aeu2003
Obteniendo datos del curso: agc209
Obteniendo datos del curso: agl253
Obteniendo datos del curso: agl255
Obteniendo datos del curso: aro105c
Obteniendo datos del curso: aro105t
Obteniendo datos del curso: aro107i
Obteniendo datos del curso: aro114m
Obteniendo datos del curso: aro115m
Obteniendo datos del curso: aro120t
Obteniendo datos del curso: aro122t
Obteniendo datos del curso: aro123t
Obteniendo datos del curso: aro124t
Obteniendo datos del curso: aro125t
Obteniendo datos del curso: ast101
Obteniendo datos del curso: ast104
Obteniendo datos del curso: arq20

In [9]:
# Convertir a DataFrame de pandas
df_cursos = pd.DataFrame(datos_cursos)

# Limpiar espacios múltiples en la columna description
df_cursos['description'] = df_cursos['description'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Guardar en CSV
df_cursos.to_csv("descripciones_cursos.csv", index=False)

### Parte II: Entrenamiento de modelo TF-DF

In [ ]:

df_cursos['sigle'] = df_cursos['sigle'].str.lower()

df_cursos['description'] = df_cursos['description'].fillna('')

tfidf = TfidfVectorizer(
    max_features=200,  # límite de palablas, con 200 funciona bastante bien
    lowercase=True,
    min_df=1
)


tfidf_matrix = tfidf.fit_transform(df_cursos['description'])

# Calcular similitud coseno entre todos los cursos
similarity_matrix = cosine_similarity(tfidf_matrix)


def recomendar_cursos(codigo_curso: str, n_recomendaciones: int = 5):
    """
    Recomienda cursos similares basado en TF-IDF
    """
    try:
    
        idx = df_cursos[df_cursos['sigle'] == codigo_curso.lower()].index[0]
        
        # Obtener similitudes del curso con todos los demás
        similitudes = similarity_matrix[idx]
        
        # Ordenar en orden descendente para obtener el de mayor similitud arriba
        indices_similares = np.argsort(similitudes)[::-1][1:n_recomendaciones+1]
        
        #crear el df
        recomendaciones = df_cursos.iloc[indices_similares][['sigle', 'name']].copy()
        recomendaciones['similaridad'] = similitudes[indices_similares]
        
        return recomendaciones
    except IndexError:
        # Si no se encuentra en curso en la lista
        print(f"Curso {codigo_curso} no encontrado")
        return None

# Uso
print("Recomendaciones para act1307:")
recomendar_cursos('act1307', n_recomendaciones=5)

Recomendaciones para act1307:


,sigle,name,similaridad
4,act1312,Performance Como Experiencia para la Transform...,0.578127
218,psg001,Lengua de Señas Chilena: la Comunidad Sorda y ...,0.552720
3,act1311,Mitos Clásicos y Creación de Relatos en la Dra...,0.542723
10,act1321,El Diálogo Teatral para la Comunicación Escénica,0.537673
153,fil2006,Introducción a la Argumentación,0.532529
